# GardenBuddy Demo

This is a set of demo notebooks to illustrate the use of the MLTE library and SDMT process, using a flower classification model.

NOTE: this demo has an additional set of requirements than MLTE. You can install them with the command: 

`poetry install --with demo`




## 1 Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. For details about setting the context, see the session.py module.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *

## 2 Build/import a `NegotiationCard`

In MLTE, we negotiation requirements with the help of a `NegotiationCard`. This can be done manually through code, but it is easier to use the MLTE UI to do so. Below we are copying a pre-built one that applies to this scenario. In MLTE, we define requirements by constructing a `NegotiationCard` that will include explicit Quality Attribute Scenarios with the requirements.

In [ ]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()


The examples below relate to a hypothetical system used by visitors to a botanical garden to identify flowers in the different gardens and learn more about them. 

In [ ]:
# Print basic system information in the negogation card to give context to the Quality attribute scenarios
print("The goals of the system are: ")
for goal in card.system.goals:
    print("  -", goal.description)
    # metric_list = ""
    # print(goal.metrics[0])
    for metric in goal.metrics:
        # metric_list = metric_list . metric.description
        print("\t as measured by: ", metric)
print("The data used/to be used in training the system is:")
for data in card.data:
    print("  -", data.description, " for ", data.purpose, " from ", data.source)

print(
    "And the usuage context for the model is that ", card.system.usage_context
)

if len(card.system.risks) > 0:
    print(
        "The stakeholder perceived risks to the system are: ",
        "\n.  -".join(card.system.risks),
    )

The following are the QASs that we want to validate through the use of MLTE:

In [ ]:
# print all quality attribute scenarios
card.print_quality_scenarios()

## 3 Define a TestSuite

In the first phase of SDMT, we define a `TestSuite` that represents the tests the completed model must will have to pass in order to be acceptable for use in the system into which it will be integrated.

In MLTE, we define the tests that will be required for the different requirements in a `TestSuite`. Note that a new `Evidence` type (`MultipleRanksums`) had to be created in this case to handle the data and `Validator` for that case, and two stand-alone `Validator`s were defined in `validators.py` to validate data using existing `Evidence` types.

From the loaded `NegotiationCard`, we can get the list of ids of its quaity attribute scenarios, that will be added to the `TestCase`s here. Those ids are the way to link the `TestCase`s to their quality attribute requirements.

Now we can create our `TestSuite`, consisting of a list of `TestCases`, each of them addressing one or more Quality Attribute Scenarios from our `NegotiationCard`. When defining the `TestCase`s below, we need to set the id of the corresponding Quality Attribute Scenario we want to test in its "quality_scenarios" attribute.

In [ ]:
from mlte.tests.test_case import TestCase
from mlte.tests.test_suite import TestSuite

# The Evidence types we will use to validate each condition.
from mlte.measurement.storage import LocalObjectSize
from mlte.measurement.cpu import LocalProcessCPUUtilization
from mlte.measurement.units import Units
from mlte.measurement.memory import LocalProcessMemoryUtilization
from mlte.evidence.types.image import Image
from mlte.evidence.types.string import String
from mlte.evidence.types.real import Real

import utils.validators as validators
from evidence.multiple_ranksums import MultipleRanksums
from mlte.evidence.types.string import String
from mlte.validation.validator import Validator


# The full test suite.
test_suite = TestSuite(
    test_cases=[
        # Fairness QAS test cases.
        TestCase(
            identifier="accuracy across gardens",
            goal="Check if model performs well accross different populations",
            quality_scenarios=["card.default-qas_001"],
            validator=validators.all_accuracies_more_or_equal_than(0.9),
        ),
        # Robustness QAS test cases.
        TestCase(
            identifier="ranksums blur2x8",
            goal="Check blur and noise for 2x8 case",
            quality_scenarios=["card.default-qas_002"],
            validator=validators.p_value_greater_or_equal_to(0.05 / 3),
        ),
        TestCase(
            identifier="ranksums blur5x8",
            goal="Check blur and noise for 5x8 case",
            quality_scenarios=["card.default-qas_002"],
            validator=validators.p_value_greater_or_equal_to(0.05 / 3),
        ),
        TestCase(
            identifier="ranksums blur0x8",
            goal="Check blur and noise for 0x8 case",
            quality_scenarios=["card.default-qas_002"],
            validator=validators.p_value_greater_or_equal_to(0.05 / 3),
        ),
        TestCase(
            identifier="effect of blur across families",
            goal="Check consistency in families",
            quality_scenarios=["card.default-qas_002"],
            validator=MultipleRanksums.all_p_values_greater_or_equal_than(
                0.05 / 141
            ),
        ),
        # Resilience QAS test case
        TestCase(
            identifier="ranksums channel loss R",
            goal="Check consistency between channel loss",
            quality_scenarios=["card.default-qas_003"],
            validator=validators.p_value_greater_or_equal_to(0.05),
        ),
        TestCase(
            identifier="ranksums channel loss G",
            goal="Check consistency between channel loss",
            quality_scenarios=["card.default-qas_003"],
            validator=validators.p_value_greater_or_equal_to(0.05),
        ),
        TestCase(
            identifier="ranksums channel loss B",
            goal="Check consistency between channel loss",
            quality_scenarios=["card.default-qas_003"],
            validator=validators.p_value_greater_or_equal_to(0.05),
        ),
        # Resource Utilization QASs test cases.
        TestCase(
            identifier="model size",
            goal="Check storage consumption",
            quality_scenarios=["card.default-qas_004"],
            validator=LocalObjectSize.get_output_type().less_than(
                150, Units.megabyte
            ),
        ),
        TestCase(
            identifier="predicting memory",
            goal="Check memory used while predicting",
            quality_scenarios=["card.default-qas_004"],
            validator=LocalProcessMemoryUtilization.get_output_type().average_utilization_less_than(
                512.0, unit=Units.megabyte
            ),
        ),
        TestCase(
            identifier="predicting cpu",
            goal="Check cpu % used while predicting",
            quality_scenarios=["card.default-qas_004"],
            validator=LocalProcessCPUUtilization.get_output_type().max_utilization_less_than(
                30.0, unit=Units.percent
            ),
        ),
        # Understandability QAS test case.
        TestCase(
            identifier="image attributions",
            goal="Check what the model is doing",
            quality_scenarios=["card.default-qas_005"],
            validator=Image.register_info("Inspect the image."),
        ),
        # Functional Correctness - Accuracy QAS test cases.
        TestCase(
            identifier="overall model accuracy",
            goal="Measure the overall accuracy of your end to end pipeline",
            quality_scenarios=["card.default-qas_006"],
            validator=Real.greater_than(0.9),
        ),
        # Functional Correctness - I/O spec QAS test cases.
        TestCase(
            identifier="input format validation success",
            goal="Model input format must conform to specified format",
            quality_scenarios=["card.default-qas_007"],
            validator=String.contains("Model - Input Validation Pass"),
        ),
        TestCase(
            identifier="output format validation success",
            goal="Model output format must conform to specified format",
            quality_scenarios=["card.default-qas_007"],
            validator=String.contains("Model - Output Validation Pass"),
        ),
        # Reliability: Input Validation QAS test cases.
        TestCase(
            identifier="input format validation error",
            goal="Model inputs must conform to specified format",
            quality_scenarios=["card.default-qas_008"],
            validator=String.contains("Model - Input Validation Error"),
        ),
        #  Analyzability QAS test cases.
        TestCase(
            identifier="detect ood inputs",
            goal="Monitor inputs for OOD data and unexpected shifts",
            quality_scenarios=["card.default-qas_009"],
            validator=String.contains("Model - Input OOD Error"),
        ),
        # Monitorability QAS test case
        TestCase(
            identifier="monitor output confidence shift",
            goal="Monitor inputs for OOD data and unexpected shifts",
            quality_scenarios=["card.default-qas_010"],
            validator=String.contains("Model - Output Confidence Error"),
        ),
        # Time Behaviour
        TestCase(
            identifier="predicting cpu time",
            goal="Check cpu time used while predicting",
            quality_scenarios=["card.default-qas_011"],
            validator=Real.less_than(2.0, Units.second),
        ),
        # Repeatability QAS test case
        TestCase(
            identifier="repeated results sampling",
            goal="Repeatedly sampling results gives same results",
            quality_scenarios=["card.default-qas_012"],
            validator=validators.p_value_greater_or_equal_to(0.05),
        ),
        # Reproducibility QAS test case
        TestCase(
            identifier="repeated training on training samples",
            goal="Repeatedly training on different sammples of training data gives same results on test data set",
            quality_scenarios=["card.default-qas_013"],
            validator=validators.p_value_greater_or_equal_to(0.05),
        ),
        # Domain Adaptability QAS test case
        TestCase(
            identifier="running in new domain",
            goal="Repeatedly training on different sammples of training data gives same results on test data set",
            quality_scenarios=["card.default-qas_014"],
            validator=MultipleRanksums.all_p_values_greater_or_equal_than(0.05),
        ),
        # Testability QAS test case
        TestCase(
            identifier="test results from dev and op env",  # test results from dev and op env
            goal="aligment of test results from dev and op environments",
            quality_scenarios=["card.default-qas_015"],
            validator=Validator.build_info_validator(
                "Inspect the alignment of results for no more than 0.25% difference."
            ),
        ),
        # Understandability QAS test case
        TestCase(
            identifier="understanding design choices",
            goal="understanding design and implementation choices",
            quality_scenarios=["card.default-qas_016"],
            validator=Validator.build_info_validator(
                "Inspect project code and documentation."
            ),
        ),
        # Maintainability QAS test case
        TestCase(
            identifier="keep ML component up to date",
            goal="keep trained ML component up to date with op environment changes",
            quality_scenarios=["card.default-qas_017"],
            validator=Validator.build_info_validator(
                "Validate work time less than 8hrs."
            ),
        ),
        # Modifiability QAS test case
        TestCase(
            identifier="update data pipelines",
            goal="aligment of test results from dev and op environments",
            quality_scenarios=["card.default-qas_018"],
            validator=Validator.build_info_validator(
                "Validate work time less than 4hrs."
            ),
        ),
        # Replaceability QAS test case
        TestCase(
            identifier="update ML training algorithm",
            goal="update ML component training algorithm",
            quality_scenarios=["card.default-qas_019"],
            validator=Validator.build_info_validator(
                "Validate train time less than 16hrs."
            ),
        ),
        # Retrainability QAS test case
        TestCase(
            identifier="retrain ML model",
            goal="update ML component through training",
            quality_scenarios=["card.default-qas_020"],
            validator=Validator.build_info_validator(
                "Model takes less than 1 hr to retrain."
            ),
        ),
        # Reuseability QAS test case
        TestCase(
            identifier="reuse ML component",
            goal="reuse ML component in new app",
            quality_scenarios=["card.default-qas_021"],
            validator=Validator.build_info_validator(
                "Validate work time less than 4hrs."
            ),
        ),
    ]
)
test_suite.save(parents=True, force=True)